In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import tensorflow as tf
import numpy as np
from ray.experimental.tfutils import TensorFlowVariables

/anaconda3/envs/flow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/anaconda3/envs/flow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1600px;height:750px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [4]:
A = np.array([[1.001, 0], [0, 0.5]])
B = np.array([[1.], [1.]])

# for controllability
cont = np.hstack([B, A@B, A@A@B])
assert np.linalg.matrix_rank(cont)==cont.shape[0]

In [5]:
x0 = np.array([[1.], [1.]])  # 2x1 vector 

A = tf.constant(np.array([[1.001, 0], [0, -0.6]]), dtype=tf.float32, name='A')  # system dynamics
B = tf.constant(np.array([[1.], [1.]]), dtype=tf.float32, name='B')

Q = tf.constant([[1.,0.], [0.,1.]], name='Q')  # weighting
R = tf.constant(1., name='R')

x = tf.constant(x0, dtype=tf.float32, name="state")

T = 10

In [6]:
####### Defining network #######
# input: state x
# output: control u

input_layer = tf.placeholder(tf.float32, (None,2), name='in_layer')
# fc1 = tf.layers.dense(inputs=input_layer, units=2,) #activation=tf.nn.sigmoid)
# fc2 = tf.layers.dense(inputs=fc1, units=2, activation=tf.nn.sigmoid)
u = tf.layers.dense(inputs=input_layer, units=1, name='u_out_layer', reuse=tf.AUTO_REUSE)

### LOSS FUNCTION ### 
loss = tf.add(tf.matmul(tf.matmul(tf.transpose(x), Q), x), 
              tf.matmul(tf.transpose(u), tf.multiply(R, u)), name='loss')

# xs = tf.identity(x, name='xs')
# us = tf.constant(0, name='us')
xs = x
us = u

# cond = lambda i, x, l, xs, us: i < T

# def body(i, x, l, xs, us):
#     next_i = i+1
#     next_x = tf.add(tf.matmul(A, x), tf.multiply(u,B))
#     next_l = tf.add(l,
#                     tf.add(tf.matmul(tf.matmul(tf.transpose(x), Q), x),
#                            tf.matmul(tf.transpose(u), tf.multiply(R, u))))
#     next_xs = tf.concat(xs, next_x)
#     next_us = tf.concat(us, u)
#     return (next_i, next_x, next_l, next_xs, next_us)

# i, xloss_f, traj_f = tf.while_loop(cond, body, 
#                                    loop_vars=[tf.constant(0), x, loss, xs, us],
#                                    shape_invariants=[tf.TensorShape([1,]), tf.TensorShape([2, 1]), 
#                                                      tf.TensorShape([1,]) , tf.TensorShape([2, None]), 
#                                                      tf.TensorShape([1, None])])
# train = tf.train.GradientDescentOptimizer(0.01).minimize(xloss_f.loss)

for i in range(T):
    # LQR loss 
#     x_term = tf.matmul(tf.matmul(tf.transpose(x), Q), x, name='x_term')
#     u_term = tf.matmul(tf.transpose(u), tf.multiply(R, u), name='u_term')
#     loss = tf.add(loss, tf.add(x_term, u_term), name='loss')  # accumulate loss
    
    # Dynamics: advancing the system dynamics
    Ax = tf.matmul(A, x, name='Ax'+str(i))
    Bu = tf.multiply(u, B, name='Bu'+str(i))  # tf.multiply because u is a scalar
    x = tf.add(Ax, Bu, name='state'+str(i))  # next state vector

    loss = tf.add(loss, tf.add(tf.matmul(tf.matmul(tf.transpose(x), Q), x), tf.matmul(tf.transpose(u), tf.multiply(R, u))), name='loss'+str(i))  # accumulate loss    
    
    u = tf.layers.dense(inputs=tf.transpose(x), units=1, name='u_out_layer', reuse=True)
    
    xs = tf.concat([xs, x], 1)
    us = tf.concat([us, u], 1)
    
opt = tf.train.GradientDescentOptimizer(0.0001)
train = opt.minimize(loss)
grads_and_vars = opt.compute_gradients(loss)

In [13]:
# show_graph(tf.get_default_graph().as_graph_def())

# sess = tf.Session()
# sess.run(tf.global_variables_initializer())
# variables = TensorFlowVariables(loss, sess)
# sess.close()
    
    
with tf.Session() as sess:    
    sess.run(tf.global_variables_initializer())
    for i in range(5):
#         sess.run(train)
        sess.run(train, feed_dict={input_layer : x0.T})
        if i % 1 == 0:
#             results = sess.run([xs, us, loss, grads_and_vars], feed_dict={input_layer : x0.T})
            results = sess.run([xs, us, loss], feed_dict={input_layer : x0.T})
            labels  = "xs us loss".split(' ')
            print('training iteration', i)
            for label,result in zip(*(labels,results)) :
                print(label)
                print(result)
                print('')
            for g, v in grads_and_vars:
                print('gradients')
                print(str(sess.run(g, feed_dict={input_layer : x0.T})) + " - " + v.name)
            print('----------------------')

training iteration 0
xs
[[ 1.          1.0583686  -0.71114707 -1.573868    0.16657186  1.9830533
   0.7047497  -2.0511045  -1.7490053   1.6986771   2.879787  ]
 [ 1.         -0.5426315  -1.4449952   0.00498736  1.7390214   0.772902
  -1.744028   -1.710142    1.3302356   2.65129    -0.41136277]]

us
[[ 0.05736851 -1.7705741  -0.86200976  1.7420138   1.8163149  -1.2802868
  -2.756559    0.30415034  3.4494314   1.1794113  -3.6177697 ]]

loss
[[82.78097]]

gradients
[[-201.90422]
 [ 468.51456]] - u_out_layer/kernel:0
gradients
[-61.22252] - u_out_layer/bias:0
----------------------
training iteration 1
xs
[[ 1.          1.0378298  -0.6792079  -1.4457271   0.1938355   1.7605584
   0.54662085 -1.7553871  -1.3376389   1.4693614   2.1743221 ]
 [ 1.         -0.5631703  -1.3801733   0.06226414  1.6036499   0.60433906
  -1.5783015  -1.3555737   1.2328479   2.0686293  -0.5376862 ]]

us
[[ 0.03682972 -1.7180755  -0.7658399   1.6410084   1.566529   -1.2156981
  -2.3025546   0.4195037   2.808338    0

## compute optimal loss with true LQR ricatti equation formulation

In [ ]:
import controlpy

In [ ]:
if type(A) != np.ndarray:  # if tensors not already evaluated
    with tf.Session() as sess:
        A, B, Q, R = sess.run([A, B, Q, R])

K, P, eig = controlpy.synthesis.controller_lqr_discrete_time(A, B, Q, R)
x = x0
u = (-K@x)
xs, us = np.array(x), np.array(u)
loss = 0
for i in range(T):
    loss += x.T@Q@x + u.T*R*u
    x = A@x + B@u
    u = (-K@x)
    xs = np.hstack([xs, x])
    us = np.hstack([us, u])